In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


Lfc = 3 # look-ahead distance
dt = 0.1  # [s]

show_animation = True


class CP: # Initial state of vehicle

    def __init__(self, x=0.0, y=3.0, angle=0.0, v=1.0):
        self.x = x
        self.y = y
        self.angle = angle
        self.v = v


def update(state, delta):
    state.angle = state.angle + delta
    state.x = state.x + state.v * math.cos(state.angle) * dt
    state.y = state.y + state.v * math.sin(state.angle) * dt
    return state

def calc_distance(x,y, cx, cy):
     dx = x - cx
     dy = y - cy
     return math.hypot(dx, dy)


def calc_goal_index(state, cx, cy):

    # search nearest point index
    dx = [state.x - icx for icx in cx]
    dy = [state.y - icy for icy in cy]
    d = [abs(math.sqrt(idx ** 2 + idy ** 2)) for (idx, idy) in zip(dx, dy)]
    ind = d.index(min(d))
    
    # check if the look ahead distance is greater than the targetpoint distance, and it should not cross the goal
    while Lfc >calc_distance(state.x,state.y,cx[ind],cy[ind]) and (ind + 1) < len(cx):
        
        ind = ind+1
        
    return ind

def pure_pursuit_control(state, cx, cy, pind):
    
        dy = cy[pind]-state.y
        ind = ((cy[pind]-state.y)/Lfc**2)
        ind1 = math.atan2(ind,1)- state.angle
        return  ind1

def main():
    #  target course
    cx = np.arange(0, 50, 0.1)
    cy = np.zeros(500)
    
    T = 200.0  # max simulation time

    # initial state
    state = CP(x=0.0, y=-3.0, angle=0.0, v=1.0)

    
    time = 0.0
    x = [state.x]
    y = [state.y]
    angle = [state.angle]
    v = [state.v]
    t = [0.0]
    
    while T >= time:
        target_ind = calc_goal_index(state, cx, cy)
   
        di = pure_pursuit_control(state, cx, cy, target_ind) #steering angle
        state = update(state, di)

        time = time + dt

        x.append(state.x)
        y.append(state.y)
        angle.append(state.angle)
        v.append(state.v)
        t.append(time)
        
        if show_animation:
            plt.cla()
            plt.plot(cx, cy, ".r", label="course")
            plt.plot(x, y, "-b", label="trajectory")
            plt.plot(cx[target_ind], cy[target_ind], "xg", label="target")
            plt.axis("equal")
            plt.grid(True)
            plt.title("Speed[km/h]:" + str(state.v * 3.6)[:4])
            plt.pause(0.001)


 

if __name__ == '__main__':
   
    main()
        